<a href="https://colab.research.google.com/github/Moore-Lab-UMass/ENCODE-API/blob/main/Pull_DNase_Bams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#Jill E. Moore
#Moore Lab - UMass Chan Medical School
#May 2022

import os
import json
import urllib
from urllib import request, parse, error
import requests

In [18]:
def Download_File(name, extension, outputDir): #downloads file to local machine
    if not os.path.exists(outputDir+"/"+name+"."+extension):
        url="https://www.encodeproject.org/files/"+name+"/@@download/"+name+"."+extension
        r = requests.get(url)
        outputFile=open(outputDir+"/"+name+"."+extension, "wb")
        outputFile.write(r.content)
        outputFile.close()

In [19]:
def Extract_Experiment_Metatdata(exp, genome): #extracts experiment metadata
  urlExp = "https://www.encodeproject.org/experiments/"+exp+"/?format=json"
  response = urllib.request.urlopen(urlExp)
  data = json.loads(response.read())

  rfa = data["award"]["rfa"] #project/phase of encode
  lab = data["lab"]["title"] #lab that submitted the experiment
  biosample = data["biosample_ontology"]["term_name"] #short name of biosample

  bam = "NA"
  spotScore = "NA"
  numReads = "NA"
  replicate = "NA"

  for entry in data["files"]: #loops through files associated with experiment
    if entry["file_type"] == "bam" and entry["status"] == "released" \
      and entry["assembly"] == genome and entry["output_type"] == "alignments":

      bam = entry["accession"] #bam file
      #Download_File(bam, "bam", "/data/output")

      replicate = ",".join([str(i) for i in entry["biological_replicates"]])
      for qc in entry["quality_metrics"]:
        if "spot1_score" in qc:
          spotScore = qc["spot1_score"]
        if "mapped" in qc:
          numReads = qc["mapped"]

      for pipeline in entry["analyses"]:
        if pipeline["title"] == "ENCODE4 v3.0.0 GRCh38":
          print(exp + "\t" + bam +"\t" + replicate + "\t" + biosample + "\t" + \
                rfa +"\t"+ lab + "\t" + str(spotScore) + "\t" + str(numReads))


In [20]:
## Parameters
genome = "GRCh38"
species = "Homo+sapiens"

## Build query
urlMain = "https://www.encodeproject.org/search/?type=Experiment&status=released" + \
    "&perturbed=false&assay_title=DNase-seq" + \
    "&replicates.library.biosample.donor.organism.scientific_name=" + species + \
    "&format=json&limit=50" #update limit=all for all datasets

response = urllib.request.urlopen(urlMain)
data = json.loads(response.read())

print("experiment_accession" + "\t" + "bam_accession" + "\t" + "replicate" "\t" + "biosample" + \
        "\t" + "rfa" +"\t"+ "lab" + "\t" + "spotScore" + "\t" + "number_reads")

for entry in data["@graph"]: #loops through experiments
  Extract_Experiment_Metatdata(entry["accession"], genome)

experiment_accession	bam_accession	replicate	biosample	rfa	lab	spotScore	number_reads
ENCSR011VHK	ENCFF528VKG	1	naive B cell	ENCODE4	John Stamatoyannopoulos, UW	0.3732	107926552
ENCSR794KUS	ENCFF900TQP	1	naive thymus-derived CD4-positive, alpha-beta T cell	ENCODE4	John Stamatoyannopoulos, UW	0.4239	123549402
ENCSR017SBI	ENCFF052ZPU	1	naive B cell	ENCODE4	John Stamatoyannopoulos, UW	0.3192	97767326
ENCSR938OKL	ENCFF114KWN	1	naive thymus-derived CD8-positive, alpha-beta T cell	ENCODE4	John Stamatoyannopoulos, UW	0.6039	134132458
ENCSR614NSH	ENCFF189WRG	1	naive thymus-derived CD4-positive, alpha-beta T cell	ENCODE4	John Stamatoyannopoulos, UW	0.454	104768802
ENCSR326TID	ENCFF677UZD	1	naive B cell	ENCODE4	John Stamatoyannopoulos, UW	0.5379	100133198
ENCSR012RCX	ENCFF223ULN	1	immature natural killer cell	ENCODE4	John Stamatoyannopoulos, UW	0.5094	124351190
ENCSR400YJR	ENCFF054KSC	1	naive thymus-derived CD8-positive, alpha-beta T cell	ENCODE4	John Stamatoyannopoulos, UW	0.4689	210376762
ENCS